In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import torch
from torch.utils.data import TensorDataset, DataLoader

In [3]:
df = pd.read_csv('/Users/dmitrykorzhov/Desktop/Root/projects/oura_ai/backend/oura_2024-04-16_2024-06-16_trends.csv')
df.columns

Index(['date', 'Sleep Score', 'Total Sleep Score', 'REM Sleep Score',
       'Deep Sleep Score', 'Sleep Efficiency Score', 'Restfulness Score',
       'Sleep Latency Score', 'Sleep Timin Score', 'Total Sleep Duration',
       'Total Bedtime ', 'Awake Time', 'REM Sleep Duration',
       'Light Sleep Duration', 'Deep Sleep Duration', 'Restless Sleep',
       'Sleep Efficiency', 'Sleep Latency', 'Sleep Timing', 'Bedtime Start',
       'Bedtime End', 'Average Resting Heart Rate',
       'Lowest Resting Heart Rate', 'Average HRV',
       'Temperature Deviation (°C)', 'Temperature Trend Deviation',
       'Respiratory Rate', 'Activity Score', 'Stay Active Score',
       'Move Every Hour Score', 'Meet Daily Targets Score',
       'Training Frequency Score', 'Training Volume Score', 'Activity Burn',
       'Total Burn', 'Steps', 'Equivalent Walking Distance', 'Inactive Time',
       'Rest Time', 'Low Activity Time', 'Medium Activity Time',
       'High Activity Time', 'Non-wear Time', 'Average

In [7]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

In [12]:
new = df.drop(columns=['date'])
mew = df.drop(columns=['Bedtime Start', 'Bedtime End'])

df_numeric = df.drop(['date', 'Bedtime Start', 'Bedtime End'], axis=1)

In [21]:
correlation = df_numeric.corr()['Deep Sleep Score'].sort_values(ascending=False)

In [22]:
correlation

Deep Sleep Score               1.000000
Deep Sleep Duration            0.938598
Previous Night Score           0.651263
Sleep Score                    0.648491
Total Sleep Score              0.607085
Rest Time                      0.565313
Sleep Timing                   0.533931
Total Sleep Duration           0.522873
Total Bedtime                  0.522170
Sleep Balance Score            0.491460
Awake Time                     0.401612
Temperature Deviation (°C)     0.401505
Respiratory Rate               0.398112
Average Resting Heart Rate     0.390347
Stay Active Score              0.361065
Restless Sleep                 0.350884
Recovery Index Score           0.321952
REM Sleep Score                0.308376
Activity Balance Score         0.289157
Sleep Timin Score              0.288499
Medium Activity Time           0.287732
Lowest Resting Heart Rate      0.271586
Readiness Score                0.259658
REM Sleep Duration             0.253643
Activity Score                 0.179257


In [25]:
features = df.drop(['Sleep Score', 'date', 'Bedtime Start', 'Bedtime End'], axis=1)
features = features.fillna(features.mean())
features_tensor = torch.tensor(features.values, dtype=torch.float32)

target = df['Sleep Score']
target = target.fillna(target.mean())
target_tensor = torch.tensor(target.values, dtype=torch.float32)

In [29]:
dataset = TensorDataset(features_tensor, target_tensor)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)